In [1]:
%cd /Users/tillschaland/Project/Gutemberg-Graph-Network/GutenbergCorpus/

/Users/tillschaland/Project/Gutemberg-Graph-Network/gutenbergCorpus


In [2]:
import pandas as pd
import  os
import definitions
import regex as re
import spacy

/Users/tillschaland/miniconda3/envs/gutenbergDBInit/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from definitions import DATA_DIR
all_books=os.listdir(DATA_DIR+'books/final/')

In [5]:
dfs=[]

for i,file in enumerate(all_books):
    df = pd.read_json(f"./data/books/final/{file}",orient='index')
    dfs.append(df)

In [6]:
def createCharacterInfo(df):
    characterlist=list()
    bookid=None
    for i,row in df.iterrows():
        characterlist.extend(list(row.PERS))
        bookid=row.bookindex

    characterlist=list(set(characterlist))
    temp_=[]
    for i,char in enumerate(characterlist):
        persdict=dict()
        persdict['sentenceBookID']=i
        persdict['name']=char
        persdict['bookID']=bookid
        temp_.append(persdict)
    return temp_

In [7]:
def createBookFixture(dfs):
    fixtures_=[] # 'bookindex','id', 'title','author','authoryearofbirth','authoryearofdeath','language','subjects'
    for i,df in enumerate(dfs):
        series=df.iloc[0]
        fixtures_.append({'pk':series.bookindex,
                          'id':series.id,
                          'title':series.title,
                          'author':series.author,
                          'authoryearofbirth':series.authoryearofbirth,
                          'authoryearofdeath':series.authoryearofdeath,
                          'language':series.language,
                          'subjects':series.subjects
                          })
        
    return fixtures_

def createSentFixture(df, bookFix, topicFix):
    temp_=[]# sentenceid(pk), bookid, sentenceText,sentIDBook, topicID(pk)
    for i,row in df.iterrows():
        try:
            topic=next(filter(lambda x: x['Topic']==row.Topic and x['Name']==row.Name,topicFix))
        except  TypeError as err:
            print("No Topic Found")            
        try:
            book=next(filter(lambda x: x['id']==row.id,bookFix))
        except TypeError as err:
            print("Noo book found")
        temp_.append(
            {
            'pk':row.sentenceIndex,
            'bookID': book['pk'],
            'sentenceText':row.sents,
            'sentIDBook':i,
            'topicID':topic['pk'],
            }
        )
    return temp_

In [8]:
for df in dfs:
    df.rename({'index':'sentenceIndex'},axis=1,inplace=True)

In [9]:
def extractChar(df):
    allChars = list()
    for i,row in df.iterrows():
        chars_ =row.PERS
        for char in chars_:
            allChars.append(char)
    return list(set(allChars))

In [10]:
def extractTopics(df):
    rel_cols=['bookindex','id','Topic','Name','Top_n_words']
    temp_ = df[rel_cols]
    listoftopics=list()
    for i,row in temp_.iterrows():
        dict_=dict()
        dict_['Topic']=row.Topic
        dict_['Name']=row.Name
        dict_['Top_n_words']=processTopics(row.Top_n_words)
        dict_['bookindex']=row.bookindex
        dict_['id']=row.id
        listoftopics.append(dict_)
    return listoftopics

def processTopics(topnwords):
    list_of_words=topnwords.split('-')
    list_of_words=[word.strip(' ') for word in list_of_words]
    return list_of_words


def removeDup(l):
    new_d = []
    for x in l:
        if x not in new_d:
            new_d.append(x)
    return new_d

def removeDictDuplicates(l):
    seen = set()
    new_l = []
    for d in l:
        t = tuple(d.items())
        if t not in seen:
            seen.add(t)
            new_l.append(d)        
    return sorted(new_l, key=lambda x: x['Topic'])   # sort by topic


In [11]:
characterTable=[]
for df in dfs:
    characterTable.extend(createCharacterInfo(df))

characterTable =  [{'pk':i, 'name':char['name'],'bookID':char['bookID'], 'sentenceBookID':char['sentenceBookID']}for i,char in enumerate(characterTable)]

In [12]:
df.head()

,sentenceIndex,bookindex,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,sents,Document,Topic,Name,Top_n_words,Probability,Representative_document,PERS
0,17952,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,,,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]
1,17953,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,[Illustration] The Enormous Room by E. E. Cumm...,[Illustration] The Enormous Room by E. E. Cumm...,3,3_room_enormous_pillars_inhabitants,room - enormous - pillars - inhabitants - eccl...,0.495287,False,[]
2,17954,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,FOUND.” He was lost by the Norton-Harjes Ambul...,FOUND.” He was lost by the Norton-Harjes Ambul...,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]
3,17955,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,to life—with the help of powerful and willing ...,to life—with the help of powerful and willing ...,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]
4,17956,3,PG8446,The Enormous Room,"Cummings, E. E. (Edward Estlin)",1894,1962,['en'],131,"{'War stories', 'Concentration camp inmates --...",NaN,the Atlantic. The following documents tell the...,the Atlantic. The following documents tell the...,-1,-1_said_time_eyes_little,said - time - eyes - little - great - say - fa...,0.000000,False,[]


In [13]:
t=removeDup(extractTopics(df))
t =  [{'pk':i, 'Topic':topic['Topic'],'Name':topic['Name'], 'top_n_word':topic['Top_n_words'],'bookindex':topic['bookindex']}for i,topic in enumerate(t)]

In [14]:
def createSent2Char(df, dfs_character):

    def helpFilter(charDict, char):
        return charDict['name']==char
    
    sent2char=[]

    for i,row in df.iterrows():
        characters=row.PERS
        for char in characters:
            filteredChar = next(filter(lambda x: helpFilter(x,char), dfs_character))
            charpk = filteredChar['pk']
            sentID = row.sentenceIndex
            sent2char.append({'charID':charpk, 'sentID':sentID})

    return sent2char

In [15]:
def isTopic(id,item):
    """
    Filter Function for filter()
    
    """
    return id==item['Topic']   


In [16]:
dfs_book=[] # 'bookindex','id', 'title','author','authoryearofbirth','authoryearofdeath','language','subjects'
dfs_sentence=[] # sentenceid(pk), bookid, sentenceText,sentIDBook, topicID(pk)
dfs_Sent2Char=[] # charID(sent.pk), sentID(sent.pk)
dfs_character=[] # id(pk),name, bookid(book(pk))
dfs_topic=[] # id(pk), bookid(book.pk), bookInternID(topicID),name, topnwords
for df in dfs:
    #df=df.reset_index().rename({'index':'sentIDBook'},axis=1)
    dfs_book.append(df[['bookindex','id', 'title','author','authoryearofbirth','authoryearofdeath','language','subjects']])
    dfs_sentence.append(df[['sentenceIndex', 'id','sents','Topic']])
    dfs_character.extend(createCharacterInfo(df))
    dfs_topic.extend(removeDup(extractTopics(df)))

characterFixture =  [{'pk':i, 'name':char['name'],'bookID':char['bookID'], 'sentenceBookID':char['sentenceBookID']}for i,char in enumerate(dfs_character)]
topicFixture =  [{'pk':i, 'Topic':topic['Topic'],'Name':topic['Name'], 'Top_n_words':topic['Top_n_words'],'bookindex':topic['bookindex']}for i,topic in enumerate(dfs_topic)]
bookFixture=createBookFixture(dfs)
sentenceFixture=[]
sent2charFixture=[]
for df in dfs:
    sent2charFixture.extend(createSent2Char(df, characterFixture))
    fitures=createSentFixture(df,bookFixture,topicFixture)
    sentenceFixture.extend(fitures)

sent2charFixture=[{'pk':i, 'charID':fix['charID'], 'sentID':fix['sentID']} for i,fix in enumerate(sent2charFixture)]

In [ ]:
list_of_Sent=list(filter(lambda x: x['fields']['charID']==15, sent2charFixture))

for sent in list_of_Sent:
    toSearch=sent['fields']['sentID']
    sentences=list(filter(lambda x: x['pk']==toSearch,sentenceFixture))

bookid=next(filter(lambda x: x['pk']==15, characterFixture))['fields']['bookID']
allTopics=list(filter(lambda x: x['fields']['bookID']==bookid, topicFixture))

distr_=dict()
for topic in allTopics:
    distr_[topic['fields']['TopicID']]= {'TopicName:':topic['fields']['TopicName'], 'Count':0}
for sent in sentences:
    distr_[sent['fields']['topicID']]['Count']+=1


print(distr_)

KeyError: 'fields'

In [24]:
def createFinalCharacterFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Character",
            "pk":fix['pk'],
            "fields": {
                'Name':fix['name'],
                'bookID':fix['bookID'],
            }
            }
        )
    return temp_

In [25]:
def createFinalTopicFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Topic",
            "pk":fix['pk'],
            "fields": {
                'TopicID':fix['Topic'],
                'TopicName':fix['Name'],
                'Top_n_words':fix['Top_n_words'],
                'bookID':fix['bookindex']
            }
            }
        )
    return temp_

In [26]:
def createFinalSent2CharFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Sent2Char",
            "pk":fix['pk'],
            "fields": {
                'charID':fix['charID'],
                'sentID':fix['sentID']
            }
            }
        )
    return temp_

In [27]:
def createFinalSentenceFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Sentence",
            "pk":int(fix['pk']),
            "fields": {
                'bookID':int(fix['bookID']),
                'sentenceText':fix['sentenceText'],
                'sentIDBook':fix['sentIDBook'],
                'topicID':fix['topicID']
            }
            }
        )
    return temp_

In [28]:
def createFinalBookFixtur(fixture):
    temp_=[]
    for fix in fixture:
        temp_.append(
            {
            "model":"webDB.Book",
            "pk":int(fix['pk']),
            "fields": {
                "bookID":fix['id'],
                "bookName":fix['title'],
                "bookauthor":fix['author'],
                "bookAuthorYearOfBirth":str(fix['authoryearofbirth']),
                "bookAuthorYearOfDeath":str(fix['authoryearofdeath']),
                "bookLanguage":fix['language'],
                "bookSubjects":fix['subjects'],
            }
            }
        )
    return temp_

In [29]:
characterFixture=createFinalCharacterFixtur(characterFixture)

In [30]:
topicFixture=createFinalTopicFixtur(topicFixture)

In [31]:
bookFixture=createFinalBookFixtur(bookFixture)

In [32]:
sentenceFixture=createFinalSentenceFixtur(sentenceFixture)

In [33]:
sent2charFixture=createFinalSent2CharFixtur(sent2charFixture)

In [ ]:
def to_int(x):
    x = x['fields']['bookID']
    print(x)
    return int(x)
sentenceFixture=[to_int(sent)for sent in sentenceFixture]

In [34]:
import json
fixtureFileNames=['character','topic','book','sentence','sent2char']
fixturesCombined=[characterFixture,topicFixture,bookFixture,sentenceFixture,sent2charFixture]
for fixture, filename in zip(fixturesCombined,fixtureFileNames):
    with open(f"./fixtures/Fixtures_Mid/{filename}FixturesSmall.json", 'w',encoding='utf-8') as f:
        json.dump(fixture,f)


# OLD